# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress

# Import API key
from api_keys import weather_api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [2]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

609

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [4]:
responses = []
for city in cities:
    print(f'Hitting OpenWeather API for {city.title()}')
    url = 'https://api.openweathermap.org/data/2.5/weather'
    params = {
        'q': city,
        'appid': weather_api_key
    }

    responses.append(requests.get(url, params))

Hitting OpenWeather API for Belmonte
Hitting OpenWeather API for Hilo
Hitting OpenWeather API for Honningsvag
Hitting OpenWeather API for Lagoa
Hitting OpenWeather API for Georgetown
Hitting OpenWeather API for Pierre
Hitting OpenWeather API for Albany
Hitting OpenWeather API for Ust-Nera
Hitting OpenWeather API for Saskylakh
Hitting OpenWeather API for Kapaa
Hitting OpenWeather API for Faanui
Hitting OpenWeather API for Tessalit
Hitting OpenWeather API for Marcona
Hitting OpenWeather API for Butaritari
Hitting OpenWeather API for Saint-Francois
Hitting OpenWeather API for Zwedru
Hitting OpenWeather API for Mahebourg
Hitting OpenWeather API for Hobart
Hitting OpenWeather API for Bluff
Hitting OpenWeather API for Clyde River
Hitting OpenWeather API for Amderma
Hitting OpenWeather API for Touros
Hitting OpenWeather API for Jamestown
Hitting OpenWeather API for Port Moresby
Hitting OpenWeather API for Broome
Hitting OpenWeather API for Gat
Hitting OpenWeather API for Darlawn
Hitting OpenW

Hitting OpenWeather API for Lugovskiy
Hitting OpenWeather API for Nouadhibou
Hitting OpenWeather API for Hambantota
Hitting OpenWeather API for Calamar
Hitting OpenWeather API for Caravelas
Hitting OpenWeather API for Laiyang
Hitting OpenWeather API for Cidreira
Hitting OpenWeather API for Goderich
Hitting OpenWeather API for Lebu
Hitting OpenWeather API for Namibe
Hitting OpenWeather API for Bambanglipuro
Hitting OpenWeather API for Saint-Philippe
Hitting OpenWeather API for Igarka
Hitting OpenWeather API for Torbay
Hitting OpenWeather API for Cercal
Hitting OpenWeather API for Sishui
Hitting OpenWeather API for Chalchihuites
Hitting OpenWeather API for Port Hardy
Hitting OpenWeather API for Yeletskiy
Hitting OpenWeather API for Kohima
Hitting OpenWeather API for Tromso
Hitting OpenWeather API for Mar Del Plata
Hitting OpenWeather API for Shubarkuduk
Hitting OpenWeather API for Warqla
Hitting OpenWeather API for Gornopravdinsk
Hitting OpenWeather API for Khasan
Hitting OpenWeather API

Hitting OpenWeather API for Chegdomyn
Hitting OpenWeather API for Chumikan
Hitting OpenWeather API for Ust-Bolsheretsk
Hitting OpenWeather API for Leh
Hitting OpenWeather API for Guaraniacu
Hitting OpenWeather API for Ishigaki
Hitting OpenWeather API for Klaksvik
Hitting OpenWeather API for Vanimo
Hitting OpenWeather API for Lavrentiya
Hitting OpenWeather API for Lautaro
Hitting OpenWeather API for Saint-Augustin
Hitting OpenWeather API for Andenes
Hitting OpenWeather API for Tuatapere
Hitting OpenWeather API for Turayf
Hitting OpenWeather API for Bud
Hitting OpenWeather API for Bay-Khaak
Hitting OpenWeather API for Sandy Bay
Hitting OpenWeather API for Goundam
Hitting OpenWeather API for Iskateley
Hitting OpenWeather API for Grand Gaube
Hitting OpenWeather API for Olavarria
Hitting OpenWeather API for Marrakesh
Hitting OpenWeather API for Naron
Hitting OpenWeather API for Vila Velha
Hitting OpenWeather API for Pacifica
Hitting OpenWeather API for Kamenskoye
Hitting OpenWeather API for

### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [9]:
cleaned_responses = []
for response in responses:
    if response.status_code == 200:
        cleaned_responses.append(response)
        
len(cleaned_responses), len(responses)

(558, 609)

In [141]:
df_rows = []
for response in responses:
    data = response.json()
    try:
        weather_col = data['weather']
        df = pd.json_normalize(weather_col)
        df1 = pd.json_normalize(data).drop('weather',axis=1)
        df = pd.concat([df, df1], axis=1)
        df = df.loc[:,~df.columns.duplicated()] # removes second id col
        df_rows.append(df)
    except KeyError:
        df = pd.json_normalize(data)
        df = df.loc[:,~df.columns.duplicated()]
        df_rows.append(df)

num_cols = []
for z in df_rows:
    num_cols.append(len(z.columns))
    
max_cols = max(num_cols)

for col in range(len(num_cols)):
    og_col_num = len(df_rows[col].columns)
    if og_col_num != max_cols:
        col_diff = max_cols - og_col_num
        for new_col in range(col_diff):
            df_rows[col][f'filler_{new_col}'] = np.nan
            
raw_df = pd.concat(df_rows)
raw_df.head()

,id,main,description,icon,base,visibility,dt,timezone,name,cod,...,filler_18,filler_19,filler_20,filler_21,filler_22,filler_23,filler_24,filler_25,filler_26,rain.1h
0,802.0,Clouds,scattered clouds,03d,stations,10000.0,1.608738e+09,-10800.0,Belmonte,200,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,803.0,Clouds,broken clouds,04n,stations,10000.0,1.608738e+09,-36000.0,Hilo,200,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,803.0,Clouds,broken clouds,04n,stations,10000.0,1.608738e+09,3600.0,Honningsvåg,200,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,802.0,Clouds,scattered clouds,03d,stations,10000.0,1.608738e+09,-3600.0,Lagoa,200,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,801.0,Clouds,few clouds,02n,stations,9000.0,1.608738e+09,28800.0,George Town,200,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [117]:
#### RAW BUILD METHOD
# raw_json = cleaned_responses[31].json()

# city_data = {}
# city_data['lon'] = [raw_json['coord']['lon']] 
# city_data['lat'] = [raw_json['coord']['lat']]
# city_data['weather_id']
# pd.DataFrame(city_data)
# data

In [ ]:
# df_rows = []
# def make_row(response):
#     # Turn OW json to df row: https://stackoverflow.com/questions/43656970/json-to-python-dataframe
    
#     data = response.json()
#     df = pd.json_normalize(data)
#     without_condensed = df.drop(columns=['weather'])
#     df_weather = pd.json_normalize(data['weather'])
    
#     df = pd.concat([df_weather, without_condensed], axis=1)
    
# #     df.columns = 
    
#     df = df.fillna(0)
#     df_rows.append(df)

# for response in cleaned_responses:
#     make_row(response)

# # pd.concat(df_rows).fillna(0)
# # d = 0
# # for i in df_rows:
# #     print(f'{d}: {len(i.columns)}')
# #     d += 1

# df_rows[31].columns

## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

In [ ]:
#  Get the indices of cities that have humidity over 100%.


In [ ]:
# Make a new DataFrame equal to the city data to drop all humidity outliers by index.
# Passing "inplace=False" will make a copy of the city_data DataFrame, which we call "clean_city_data".


## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

## Latitude vs. Humidity Plot

## Latitude vs. Cloudiness Plot

## Latitude vs. Wind Speed Plot

## Linear Regression

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression